# YOLACT
Inferencia en imagenes y video con [YOLACT](https://github.com/dbolya/yolact).

### Runtime Setup
- Ir a Runtime > Change Runtime Type
- Seleccionar GPU (TPU won't work)

### YOLACT
YOLACT es un algoritmo de segmentación de objetos detallado en los siguientes papers:  
- [YOLACT: Real-time Instance Segmentation](https://arxiv.org/abs/1904.02689)
- [YOLACT++: Better Real-time Instance Segmentation](https://arxiv.org/abs/1912.06218)

**Muchas gracias a los autores: Daniel Bolya, Chong Zhou, Fanyi Xiao, Yong Jae Lee!**

Este notebook fue creado por Adam Kelly, instructor/creador de [immersivelimit.com](https://www.immersivelimit.com).

# Setup inicial

Primero, instalamos algunas librerías Python. Colab probablemente ya tenga estos requerimientos, pero mejor incluirlos antes de no encontrarlos después.

## Para hacer instance o segmentation no solo se necesita los BB (Bounding Boxes) como Ground True, sino también los pixeles que encierran cada elemento. 

In [1]:
# Cython necesita ser instalado antes de pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib gdown
!pip install torch===1.4.0
!pip install torchvision===0.5.0

     |████████████████████████████████| 753.4MB 22kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
     |████████████████████████████████| 4.0MB 11.1MB/s 
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


## Clonar YOLACT desde GitHub

In [2]:
# Nos aseguramos de estar en el folder raíz
%cd /content

!git clone https://github.com/dbolya/yolact.git

/content
Cloning into 'yolact'...
remote: Enumerating objects: 2936, done.
remote: Total 2936 (delta 0), reused 0 (delta 0), pack-reused 2936
Receiving objects: 100% (2936/2936), 21.20 MiB | 9.54 MiB/s, done.
Resolving deltas: 100% (2002/2002), done.


## DCNv2
DCNv2 es una librería externa necesaria.

In [3]:
# Cambiar al directorio correcto
%cd /content/yolact/external/DCNv2

# Build DCNv2
!python setup.py build develop

/content/yolact/external/DCNv2
running build
running build_ext
building '_ext' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/content
creating build/temp.linux-x86_64-3.6/content/yolact
creating build/temp.linux-x86_64-3.6/content/yolact/external
creating build/temp.linux-x86_64-3.6/content/yolact/external/DCNv2
creating build/temp.linux-x86_64-3.6/content/yolact/external/DCNv2/src
creating build/temp.linux-x86_64-3.6/content/yolact/external/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.6/content/yolact/external/DCNv2/src/cuda
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/content/yolact/external/DCNv2/src -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include -I/usr/local/lib/python3.6/dist-packages/torch/include/TH -I/usr/local/

## Pesos Pre entrenados


In [4]:
# Nos aseguramos de estar en el folder raíz
%cd /content

# Clonar el repositorio
!git clone https://github.com/chentinghao/download_google_drive.git

# Crear un folder nuevo para los pesos pre entrenados
!mkdir -p /content/yolact/weights

# Descargar archivo
!python ./download_google_drive/download_gdrive.py 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP ./yolact/weights/yolact_plus_resnet50_54_800000.pth

/content
Cloning into 'download_google_drive'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
129MB [00:01, 75.2MB/s]


# Imagenes de test

In [5]:
# Crear folder nuevo para las imagenes de test y cambiar
!mkdir /content/test_images
%cd /content/test_images

# Lista de URLs de COCO dataset
get_imgs = ['http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg',
            'http://farm3.staticflickr.com/2835/9520716824_51220b13d0_z.jpg',
            'http://farm3.staticflickr.com/2817/10105052924_0c8d79e62c_z.jpg',
            'https://farm3.staticflickr.com/2637/3679968895_37a79e55b3_z.jpg',
            'https://farm8.staticflickr.com/7137/7705349298_6bea29cf1c_z.jpg',
            'http://farm5.staticflickr.com/4105/4981945644_3f162314ed_z.jpg',
            'http://farm1.staticflickr.com/40/101757885_a8fc40e273_z.jpg',
            'http://farm3.staticflickr.com/2654/3897535041_fa8dd5570b_z.jpg']

# Descargar las imagenes
for img_url in get_imgs:
  !wget {img_url} -nc

# Listar el contenido el folder
!pwd
!ls

/content/test_images
--2020-11-08 16:42:40--  http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg
Resolving farm8.staticflickr.com (farm8.staticflickr.com)... 52.84.229.90, 2600:9000:200a:2c00:0:5a51:64c9:c681, 2600:9000:200a:c200:0:5a51:64c9:c681, ...
Connecting to farm8.staticflickr.com (farm8.staticflickr.com)|52.84.229.90|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg [following]
--2020-11-08 16:42:40--  https://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg
Connecting to farm8.staticflickr.com (farm8.staticflickr.com)|52.84.229.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘6946559805_7edf757660_z.jpg’

6946559805_7edf7576     [ <=>                ]  52.94K  --.-KB/s    in 0.001s  

2020-11-08 16:42:40 (61.4 MB/s) - ‘6946559805_7edf757660_z.jpg’ saved [54206]

--2020-11-08 16:42:40--  http:

# Ejecutar la inferencia (predicción) sobre las imagenes


In [6]:
# Regresamos al directorio raíz
%cd /content

# Eliminar el folder output_images
!rm /content/output_images -r

# Crear un folder nuevo output images
!mkdir -p /content/output_images

# Ejecutar la inferencia con los pesos pre entrenados en todas las imágenes
!python ./yolact/eval.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth --config=yolact_plus_resnet50_config --score_threshold=0.15 --top_k=15 --images=test_images:output_images

/content
rm: cannot remove '/content/output_images': No such file or directory
Loading model... Done.

test_images/3679968895_37a79e55b3_z.jpg -> output_images/3679968895_37a79e55b3_z.png
test_images/101757885_a8fc40e273_z.jpg -> output_images/101757885_a8fc40e273_z.png
test_images/7705349298_6bea29cf1c_z.jpg -> output_images/7705349298_6bea29cf1c_z.png
test_images/6946559805_7edf757660_z.jpg -> output_images/6946559805_7edf757660_z.png
test_images/3897535041_fa8dd5570b_z.jpg -> output_images/3897535041_fa8dd5570b_z.png
test_images/10105052924_0c8d79e62c_z.jpg -> output_images/10105052924_0c8d79e62c_z.png
test_images/4981945644_3f162314ed_z.jpg -> output_images/4981945644_3f162314ed_z.png
test_images/9520716824_51220b13d0_z.jpg -> output_images/9520716824_51220b13d0_z.png
Done.


## Mostrar la imágenes

In [7]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

output_images = Path('output_images')

def show_image(img_path):
  img = cv2.imread(img_path)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.show()

for img_path in output_images.iterdir():
  print(img_path)
  show_image(str(img_path))

Output hidden; open in https://colab.research.google.com to view.

# Procesando un video

In [8]:
import gdown

!mkdir -p /content/test_video

url = 'https://drive.google.com/uc?id=1s5TeREqsZNT3M9HVVOp1hM_mGnWAVRRR'
output = '/content/test_video/vid1.MP4'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1s5TeREqsZNT3M9HVVOp1hM_mGnWAVRRR
To: /content/test_video/vid1.MP4
12.0MB [00:00, 27.9MB/s]


'/content/test_video/vid1.MP4'

In [9]:
%cd /content
!mkdir -p /content/output_videos
output_path = "/content/output_videos/video_output.mp4"
!python ./yolact/eval.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth --score_threshold=0.15 --top_k=15 --video_multiframe=1 --video={output}:{output_path}

/content
Config not specified. Parsed yolact_plus_resnet50_config from the file name.

Loading model... Done.
Initializing model... Done.

Processing Frames  ██████████████████████████████    232 /    232 (100.00%)    12.97 fps        
